# Pcap analysis

In [17]:
import pandas as pd
from scapy.all import *
from scapy.contrib.coap import *
from scapy.contrib.mqtt import *
from collections import Counter

In [18]:
packets = rdpcap('capture.pcapng')

CQ1) How many different Confirmable PUT requests obtained an 
unsuccessful response from the local CoAP server?

In [19]:
# Select Confirmable PUT request
put_con_packets = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code == 3 and pkt[CoAP].type == 0]

# Select only requests with unsuccessful responce from the local CoAP server
uns_resp = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code > 69 and pkt["IP"].src == "127.0.0.1"]

count = len({pkt[CoAP].token for pkt in put_con_packets} & {pkt[CoAP].token for pkt in uns_resp})

print("Answer:")
print(count)

Answer:
22


CQ2) How many CoAP resources in the coap.me public server received the 
same number of unique Confirmable and Non Confirmable GET requests?

Assuming a resource receives X different CONFIRMABLE requests and Y different 
NONCONFIRMABLE  GET requests, how many resources have X=Y, with X>0?

In [20]:
con_resources = Counter()
non_resources = Counter()

for pkt in packets: 
    if pkt.haslayer(CoAP) and pkt[CoAP].code == 1 and pkt["IP"].dst == "134.102.218.18":
        uri_path_str = ""
        for option in pkt[CoAP].options:
            if option[0] == "Uri-Path":
                uri_path = option[1].decode('utf-8')
                if uri_path:
                    uri_path_str = uri_path_str + "/" + uri_path
        if pkt[CoAP].type == 0:    
            con_resources[uri_path_str]+=1
        else:
            non_resources[uri_path_str]+=1
            

result = [r for r in (set(con_resources.keys()) & set(non_resources.keys())) if con_resources[r] == non_resources[r]]

count = len(result)

print(result)

print("Answer:")
print(count)

['/validate', '/large', '/secret']
Answer:
3


CQ3) How many different MQTT clients subscribe to the public broker 
HiveMQ using multi-level wildcards?

In [23]:
list = []
for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 8 and (pkt.dst == "35.158.43.69" or pkt.dst == "35.158.34.213" or pkt.dst == "18.192.151.104"):
        if len(pkt[MQTT].topics) == 1:
            if "#" in pkt[MQTT].topics[0].topic.decode("utf-8"):
                list.append(pkt[TCP].sport)
        else:
            if "#" in pkt[MQTT].topics[1].load.decode("utf-8"):
                list.append(pkt[TCP].sport)

result = set(list)
print(result)
print("Answer:")
print(len(result))

{38641, 38619, 54449, 57863}
Answer:
4


CQ4) How many different MQTT clients specify a last Will Message to be 
directed to a topic having as first level "university"?

In [ ]:
list = []
for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 1:
        try:
            if pkt[MQTT].willflag == 1:
                if pkt[MQTT].willtopic.decode("utf-8").startswith("university/"):
                    list.append(pkt[TCP].sport)
        except:
            mqtt_payload = pkt[MQTT].load # Ottieni il payload MQTT come bytes
        
            if len(mqtt_payload) > 9:  # Il byte dei flag deve esistere
                connect_flags = mqtt_payload[9]  # Il secondo byte del pacchetto è il flag
                
                will_flag = (connect_flags & 0b00000100) >> 2  # Estrai il 3° bit
                
                if will_flag == 1:
                    if pkt[MQTT].willtopic.decode("utf-8").startswith("university/"):
                        list.append(pkt[TCP].sport)

result = set(list)
print(result)
print("Answer:")
print(len(result))

{38083}
Answer:
1


CQ5) How many MQTT subscribers receive a last will message derived from 
a subscription without a wildcard?

CQ6) How many MQTT publish messages directed to the public broker 
mosquitto are sent with the retain option and use QoS “At most once”?

CQ7) How many MQTT-SN messages on port 1885 are sent by the clients to 
a broker in the local machine?